In [1]:
import os
from dotenv import load_dotenv, find_dotenv 
_ = load_dotenv(find_dotenv()) 
openai_api_key = os.environ["OPENAI_API_KEY"] 

#### **RunnablePassthrough**

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [3]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

In [4]:
chain.invoke("Hitesh")

'Hitesh'

#### **Runnable Lambda**

In [5]:
def russian_lastname(name : str) -> str:
    return f"{name}ovich"

In [6]:
from langchain_core.runnables import RunnableLambda 

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

In [7]:
chain.invoke("Hitesh")

'Hiteshovich'

#### **Runnable Parallel**

In [8]:
from langchain_core.runnables import RunnableParallel 

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(), 
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [9]:
chain.invoke("Hitesh")

{'operation_a': 'Hitesh', 'operation_b': 'Hiteshovich'}

In [10]:
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.prompts import ChatPromptTemplate 

In [11]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {guy}")

output_parser = StrOutputParser() 

In [12]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich" 

In [13]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(), 
        "guy": RunnableLambda(russian_lastname_from_dictionary), 
        "operation_c": RunnablePassthrough()
    }
) | prompt | model | output_parser

In [14]:
chain.invoke({
    "name": "Dave"
})

'Daveovich is actually a fictional character created by author J.D. Salinger in his short story "A Girl I Knew." The character is known for his eccentric and unpredictable behavior, making him a memorable figure in Salinger\'s works.'

In [15]:
from langchain_community.vectorstores import FAISS 
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel 
from langchain_openai import ChatOpenAI, OpenAIEmbeddings 

vector_store = FAISS.from_texts(
    ["The primary reason for applying for this job stems from my earnest desire to gain practical experience and further enhance my skills in Python. I am deeply passionate about Machine Learning, Deep Learning and Data Science and eager to immerse myself in real-world projects, working alongside seasoned professionals. This job presents an invaluable opportunity for me to translate theoretical knowledge into tangible outcomes, while also allowing me to contribute meaningfully to your team. I am enthusiastic about the chance to learn and grow within your esteemed organization, and I am committed to making the most of this valuable learning experience."], embedding=OpenAIEmbeddings()
)

In [16]:
retrievers = vector_store.as_retriever()

In [17]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [18]:
prompt = ChatPromptTemplate.from_template(template)

In [19]:
model = ChatOpenAI(model = "gpt-3.5-turbo")

In [20]:
retreival_chain = (
    RunnableParallel({"context": retrievers, "question": RunnablePassthrough()}) 
    | prompt 
    | model 
    | StrOutputParser()
)

In [21]:
retreival_chain.invoke("I am enthusiastic about ?")

'learning and growing within the esteemed organization and making the most of the valuable learning experience.'

In [26]:
from operator import itemgetter 

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.runnables import RunnablePassthrough 
from langchain_openai import ChatOpenAI, OpenAIEmbeddings 

model = ChatOpenAI(model = "gpt-3.5-turbo")

vectorstore = FAISS.from_texts(
    ["The primary reason for applying for this job stems from my earnest desire to gain practical experience and further enhance my skills in Python. I am deeply passionate about Machine Learning, Deep Learning and Data Science and eager to immerse myself in real-world projects, working alongside seasoned professionals. This job presents an invaluable opportunity for me to translate theoretical knowledge into tangible outcomes, while also allowing me to contribute meaningfully to your team. I am enthusiastic about the chance to learn and grow within your esteemed organization, and I am committed to making the most of this valuable learning experience."], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever() 

template = """Answer the question based only on the following context: {context}
Question : {question}

Answer in the following language: {language} 
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context" : itemgetter("question") | retrievers, 
        "question" : itemgetter("question"), 
        "language" : itemgetter("language"), 
    }
    | prompt 
    | model 
    | StrOutputParser() 
)

In [28]:
chain.invoke({"question":"Why this job presents an invaluable opportunity for me", "language" : "spanish"})

'Este trabajo presenta una oportunidad invaluable para mí porque me permitirá adquirir experiencia práctica y mejorar mis habilidades en Python. También me brindará la oportunidad de sumergirme en proyectos del mundo real, trabajar junto a profesionales experimentados y contribuir significativamente al equipo.'